In [222]:
from sgp4.api import jday
import datetime
import pandas as pd
from sgp4.api import Satrec
from sgp4 import omm
import numpy as np
%matplotlib notebook
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
from sklearn.neighbors import KernelDensity
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score


In [223]:
#Get date
current_time = datetime.datetime.now()
xd, xr = jday(current_time.year , current_time.month , current_time.day, current_time.hour, current_time.minute , current_time.second)
print(xd)
print(xr)

2460422.5
0.5910648148148148


In [224]:
with open('../Data/satellitedata.csv') as f:
    lines = omm.parse_csv(f)
    fields = [field for field in lines]
 

In [225]:
Satellite = []
result_satellite = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)    
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    if any([abs(coord) > 10_000 for coord in r]):
     continue
    Satellite.append(r)
    result_satellite.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    
  

data_satellite = np.array(Satellite)
data_satellite = data_satellite[~np.isnan(data_satellite).any(axis=1)]

result_satellite = pd.DataFrame(result_satellite,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2],
                                   mode='markers',name="Satellite", marker={"size":12, "color":"white"})])

fig.show()

In [226]:
normalization = np.linalg.norm(data_satellite, axis=1)[:, np.newaxis]
data_satellite_norm = data_satellite / normalization

fig = go.Figure(data=[go.Scatter3d(x=data_satellite_norm[:,0], y=data_satellite_norm[:,1], z=data_satellite_norm[:,2],
                                   mode='markers',name="Satellite", marker={"size":10, "color":"white"})])

fig.show()

In [227]:
# DBSCAN for cosmos garbage
with open('../Data/cosmos.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Cosmos_G = []
result_cosmos = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Cosmos_G.append(r)
    result_cosmos.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])

data_garbage_cosmos = np.array(Cosmos_G)
data_garbage_cosmos = data_garbage_cosmos[~np.isnan(data_garbage_cosmos).any(axis=1)]

result_cosmos = pd.DataFrame(result_cosmos,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers')])
fig.show()


model = DBSCAN(eps=800, min_samples=4)
model.fit_predict(data_garbage_cosmos)
pred_cosmos = model.fit_predict(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":pred_cosmos})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()


In [228]:
print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

number of cluster found: 45
cluster for each point:  [ 0 -1  1  0  0  2  3  1  1  4  0  5 -1 41 -1 36 -1 -1  0 10  6  0  0  7
  0 -1 -1 -1  0  7 -1  1  8  0 -1  1 -1  0  9  0 -1 -1 37 10 -1 11 12 -1
  7 -1  0 -1 15 28 -1 -1 13 -1 14 -1  7 -1 15 -1 -1 -1 -1 -1 -1 36 -1  0
 -1  5  1 -1 -1 16 -1  1  7 -1 -1  1  0 12  0  0  7  3 -1 -1  4 -1 42  7
 -1 14  3 17  0  0 -1  0 41  3 13  9  0 -1  0 -1  7  0 18 16  5 -1 40  0
 -1  1 -1  9  6  0 -1  1 -1 19 -1  0 -1  1  6 20 -1  0 14 -1  0 13 -1 21
 27 -1 -1 22 29  2 -1 23 24 -1 -1 17 25 -1  7  1  0  1 -1 13 26 15 21 -1
 -1 -1  0 27 -1  7 -1 -1 41  0 20 16 -1  0 -1 28  0 -1 20  0 29  0 -1  0
 30 29 -1 -1  6 13 30 -1 18 -1  0  6  1 -1  7 -1  0 -1  1 -1 17 21 33  0
  7 -1  7  5 -1  0 28  0 -1  7 -1 16  1  7 11  0  6 -1  7  0  7  0  6  1
 -1 28  8 31 -1  0  0  1  6  7 -1 -1  0 30 22  9 -1  0  6  2  3 -1  0  0
  1 31  0 32  0 10 21 -1  1 33 42  6  9 -1 11 42 31 -1  7  1 -1 -1 20  1
 -1  1 -1 16 -1  0 10 -1  7  7  0 -1  0 -1  7 -1 -1 -1 37 10 -1 25 -1  

In [229]:
# DBSCAN for iridium garbage
with open('../Data/iridium.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Iridium_G = []
result_iridium = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Iridium_G.append(r)
    result_iridium.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    

data_garbage_iridium = np.array(Iridium_G)
data_garbage_iridium = data_garbage_iridium[~np.isnan(data_garbage_iridium).any(axis=1)]


result_iridium = pd.DataFrame(result_iridium,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()


fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers')])
fig.show()

model = DBSCAN(eps=1570, min_samples=4)
model.fit_predict(data_garbage_iridium)
pred_iridium = model.fit_predict(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":pred_iridium})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()

In [230]:
print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

number of cluster found: 9
cluster for each point:  [ 0  1  2 -1  3  1 -1  4  1 -1  1 -1  4 -1  1  0  1  6 -1  1  1  1 -1  5
  0  5 -1  3 -1  5 -1 -1 -1  7 -1 -1  0  0  0  3  7 -1 -1  1 -1  6  7 -1
 -1  1 -1 -1 -1  0  0  1  1 -1  0  1  6  1  1 -1  0 -1 -1 -1  0  3  0  3
 -1  1 -1  0  0 -1  0  1  2  7  1  0  0 -1  0 -1  1 -1  5 -1  3  1  1  0
  5  0 -1 -1  6  1 -1  0 -1  5 -1 -1  0  0  2  5  1 -1 -1  0 -1  1  0  0
  1 -1  1  5  2 -1 -1  1 -1  5 -1 -1  0  5 -1  4  3  0  1 -1  5  1 -1  1
 -1  1  0 -1 -1 -1  1  1  4  1 -1  0  0  5  0  5  5  0  5 -1  1]


In [231]:
#KDE for iridium garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1500)
model.fit(data_garbage_iridium)
log_dens_iridium = model.score_samples(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_dens_iridium})])
fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()

In [232]:
#KDE for cosmos garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1000)
model.fit(data_garbage_cosmos)
log_dens_cosmos = model.score_samples(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_dens_cosmos})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()


In [233]:
# EM Algorithm for data_garbage_cosmos


# Filter out rows with inf or NaN values
data_garbage_cosmos = np.array([row for row in data_garbage_cosmos if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_cosmos), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_cosmos, mu[k], sigma[k])
        z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_cosmos, axis=0) / weight
        diff = data_garbage_cosmos - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_cosmos) + n_components * 1e-6), 0, 1)

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data')])
fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [234]:
# EM Algorithm for data_garbage_iridium


# Filter out rows with inf or NaN values
data_garbage_iridium = np.array([row for row in data_garbage_iridium if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_iridium), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_iridium, mu[k], sigma[k])
        z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_iridium, axis=0) / weight
        diff = data_garbage_iridium - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_iridium) + n_components * 1e-6), 0, 1)

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:, 0], y=data_garbage_iridium[:, 1], z=data_garbage_iridium[:, 2], mode='markers', name='Data')])
fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [265]:
#Calinski Harabasz for Cosmos
scores = []

for i in range(10, 21):
    kmeans = KMeans(n_clusters=i).fit(data_garbage_cosmos)
    
    score = calinski_harabasz_score(data_garbage_cosmos, kmeans.labels_)
    
    scores.append(score)
    
    
for i,score in enumerate(scores):
    print(f"n_clusters={i+10}, score={score}")

highest_score_index = scores.index(max(scores))

print(f"The highest score is {max(scores)} at i = {highest_score_index + 10}")

n_clusters=10, score=415.54045982320457
n_clusters=11, score=416.8380596474159
n_clusters=12, score=408.8017160572509
n_clusters=13, score=433.8188330162922
n_clusters=14, score=408.5624786039409
n_clusters=15, score=418.78774270687626
n_clusters=16, score=426.06548404783234
n_clusters=17, score=409.32685708919195
n_clusters=18, score=432.3941288933604
n_clusters=19, score=427.07202127856533
n_clusters=20, score=423.4802885011121
The highest score is 433.8188330162922 at i = 13


In [236]:
#K-Means for data_garbage_cosmos
kmeans = KMeans(n_clusters=20).fit(data_garbage_cosmos)


labels = kmeans.labels_

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data', marker=dict(size=8, color=labels, colorscale='Viridis'))])

fig.add_trace(go.Scatter3d(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], z=kmeans.cluster_centers_[:, 2],
                          mode='markers', name='Cluster Centers', marker=dict(size=12, color='red', opacity=0.7)))

fig.show()

In [256]:
#Calinski Harabasz for iridium
scores_for_iridium = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i).fit(data_garbage_iridium)
    
    score = calinski_harabasz_score(data_garbage_iridium, kmeans.labels_)
    
    scores_for_iridium.append(score)
   

for i, score in enumerate(scores_for_iridium):
    print(f"n_clusters={i+2}, score={score}")

highest_score_index = scores_for_iridium.index(max(scores_for_iridium))

print(f"The highest score is {max(scores_for_iridium)} at i = {highest_score_index + 2}")

n_clusters=2, score=129.6588767998755
n_clusters=3, score=108.84605107226538
n_clusters=4, score=119.98209720257208
n_clusters=5, score=111.92832215171263
n_clusters=6, score=98.28995873487163
n_clusters=7, score=107.72134203639713
n_clusters=8, score=108.75521132304739
n_clusters=9, score=111.14383468919041
n_clusters=10, score=115.97115175533436
The highest score is 129.6588767998755 at i = 2


In [238]:
#K-means for data_garbage_iridium
kmeans = KMeans(n_clusters=6).fit(data_garbage_iridium)

labels = kmeans.labels_
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":labels})])

fig.add_trace(go.Scatter3d(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], z=kmeans.cluster_centers_[:, 2],
                          mode='markers', name='Cluster Centers', marker=dict(size=12, color='red', opacity=0.7)))

fig.show()

In [239]:
result_iridium['Cluster'] = pred_iridium
result_iridium['Density'] = log_dens_iridium
result_iridium.to_csv('../Data/result_iridium.csv', index= False)
result_iridium

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,IRIDIUM 33,1997-051C,2024-04-22T02:47:54.790368,1437.307959,-1792.449309,-6785.150693,0,-27.581669
1,IRIDIUM 33 DEB,1997-051L,2024-04-21T22:40:25.916160,-1400.750265,2063.292396,6681.976923,1,-27.682796
2,IRIDIUM 33 DEB,1997-051N,2024-04-21T13:07:08.510592,2486.924528,-2935.825656,6007.185048,2,-28.027518
3,IRIDIUM 33 DEB,1997-051P,2024-04-21T22:07:21.188928,6101.872410,-3371.079625,-1563.022514,-1,-28.999688
4,IRIDIUM 33 DEB,1997-051Q,2024-04-21T22:20:10.726944,-2868.942597,6040.283365,-2432.498138,3,-28.098046
...,...,...,...,...,...,...,...,...
160,IRIDIUM 33 DEB,1997-051ABX,2024-04-21T19:40:25.487616,1841.833921,4649.620824,-5024.931865,5,-28.378584
161,IRIDIUM 33 DEB,1997-051ACD,2024-04-12T03:58:25.077792,-1658.411028,-2808.936172,-6268.980482,0,-27.330206
162,IRIDIUM 33 DEB,1997-051ACE,2024-04-21T19:05:07.279296,-1926.398346,3025.204712,-6154.732059,5,-27.640617
163,IRIDIUM 33 DEB,1997-051ACG,2024-04-16T10:06:52.067808,6003.309793,37.097516,-3798.463849,-1,-29.188301


In [240]:
result_cosmos['Cluster'] = pred_cosmos
result_cosmos['Density'] = log_dens_cosmos
result_cosmos.to_csv('../Data/result_cosmos.csv', index= False)
result_cosmos

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,COSMOS 2251,1993-036A,2024-04-21T22:45:46.764288,2735.973737,613.490205,6593.571034,0,-27.345820
1,COSMOS 2251 DEB,1993-036E,2024-04-21T18:35:57.065856,4889.630986,-3323.194261,-4071.422475,-1,-28.193396
2,COSMOS 2251 DEB,1993-036F,2024-04-21T20:53:40.875072,-1947.282520,299.608446,-6866.131068,1,-27.321581
3,COSMOS 2251 DEB,1993-036G,2024-04-22T02:39:37.381248,-1555.191966,-1699.247301,6646.167757,0,-27.365035
4,COSMOS 2251 DEB,1993-036H,2024-04-21T21:34:11.502336,1971.083417,520.080136,6852.807203,0,-27.229561
...,...,...,...,...,...,...,...,...
765,COSMOS 2251 DEB,1993-036BXS,2024-04-13T09:41:25.155168,7190.187945,-1629.035515,1486.915271,-1,-28.901600
766,COSMOS 2251 DEB,1993-036BXY,2024-04-17T02:49:01.027200,-1692.576519,-6381.561177,2667.954262,-1,-28.155963
767,COSMOS 2251 DEB,1993-036BYA,2024-04-20T02:14:41.174304,-865.728241,2814.082965,-6365.726898,1,-27.354470
768,COSMOS 2251 DEB,1993-036BYD,2024-04-16T10:23:21.480000,3368.588121,-2811.385471,5892.297837,0,-27.604596


In [241]:
result_satellite.to_csv('../Data/result_satellite.csv', index= False)
result_satellite

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z
0,SOYUZ-MS 25,2024-055A,2024-04-21T11:51:22.943520,-3766.662047,-5602.932392,744.210272
1,STARLINK-31463,2024-056A,2024-04-22T04:00:00.999936,296.416282,-4984.640605,4639.899820
2,STARLINK-31487,2024-056B,2024-04-22T04:00:00.999936,3660.951890,-3970.500914,4120.379330
3,STARLINK-31386,2024-056C,2024-04-21T22:00:00.999936,3462.338139,-4073.455148,4191.130974
4,STARLINK-31409,2024-056D,2024-04-21T22:00:00.999936,3847.938625,-3867.778466,4048.431127
...,...,...,...,...,...,...
203,KASHIWA,1998-067WH,2024-04-21T10:05:53.741760,1354.338573,-4487.197267,4896.253991
204,2024-069D,2024-069D,2024-04-18T08:56:53.132352,-5263.651721,3724.074403,-1298.546937
205,2024-072A,2024-072A,2024-04-22T04:20:31.316928,-346.811275,852.859854,6810.403663
206,2024-075A,2024-075A,2024-04-21T07:31:09.528672,2355.977860,5180.563913,-3850.695477


In [242]:
result_iridium.groupby('Cluster')['Density'].mean().abs()

Cluster
-1    28.699444
 0    27.552266
 1    27.544352
 2    28.048525
 3    28.149568
 4    28.285562
 5    27.803493
 6    28.059249
 7    28.268803
Name: Density, dtype: float64

In [243]:
result_cosmos.groupby('Cluster')['Density'].mean().abs()

Cluster
-1     28.348842
 0     27.345707
 1     27.396144
 2     28.116233
 3     28.275656
 4     27.718479
 5     28.249083
 6     27.590221
 7     27.581497
 8     28.013016
 9     27.872122
 10    27.796716
 11    28.127511
 12    28.064381
 13    28.001879
 14    28.191841
 15    28.252392
 16    27.797492
 17    27.880585
 18    27.889949
 19    27.853828
 20    28.095667
 21    27.957840
 22    28.146605
 23    27.815976
 24    28.113423
 25    28.021858
 26    27.884860
 27    28.211594
 28    27.554003
 29    28.072457
 30    27.886486
 31    28.373469
 32    28.022417
 33    27.831277
 34    27.519409
 35    28.155573
 36    28.106425
 37    28.089916
 38    28.256122
 39    27.893515
 40    28.373021
 41    28.208171
 42    28.505555
 43    27.824982
Name: Density, dtype: float64

In [244]:
distances_s_i = []
for i, garbage in enumerate(data_garbage_iridium):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_i.append((i, j, distance))

distances = pd.DataFrame(distances_s_i, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_i.csv', index=False)

       garbage_index  satellite_index      distance
0                  0                0   9914.250435
1                  0                1  11917.361215
2                  0                2  11341.035500
3                  0                3  11392.211994
4                  0                4  11290.909014
...              ...              ...           ...
34315            164              203   7875.274078
34316            164              204   8335.750233
34317            164              205   2407.701509
34318            164              206  11377.958871
34319            164              207   9081.554950

[34320 rows x 3 columns]


In [245]:
distances_s_c = []
for i, garbage in enumerate(data_garbage_cosmos):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_c.append((i, j, distance))

distances = pd.DataFrame(distances_s_c, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_c.csv', index=False)

        garbage_index  satellite_index      distance
0                   0                0  10730.480109
1                   0                1   6411.500632
2                   0                2   5290.107406
3                   0                3   5316.651222
4                   0                4   5272.188536
...               ...              ...           ...
160155            769              203   6233.120406
160156            769              204  11002.374570
160157            769              205   2501.651060
160158            769              206  11083.901103
160159            769              207  11564.623687

[160160 rows x 3 columns]
